# Assignment 1, Extragalactic Astronomy
Nasser Mohammed

Feburary 3rd, 2025

> The purpose of this question is to familiarize you with the basics of luminosity functions and with the relationship between the luminosity function, luminosity density, and mass density.
> Assume that galaxies can be described by a Schechter function luminosity function given in Table 4 of Loveday et al. 2015, MNRAS, 451, 2, 1540-1552.

## 1.1

> Write some code to compute the local number density of blue and red galaxies in the absolute magnitude range -23 < Mr < -14. Supply both the code and the answer. Any software or language is fine, though all things being equal I recommend a Jupyter notebook with Python for simplicity. In any case, I will zing you mercilessly if you don’t comment your notebook (or code file) appropriately to make it clear what you are doing.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

The Schecter function in units of number per magnitude interval is:

$$n(M) \ dM = 2/5 \ \phi_* \ln(10) \ [10^{0.4 (M_* - M)}]^{\alpha+1} \ \text{exp}[-10^{0.4(M_*-M)}]$$

To find the number per Mpc -3 for the provided magnitude interval we would integrate over it:

$$\int_{-23}^{-14} n(M) \ dM

In [30]:
h = 0.7 # dimensionless hubble constant 
MStar_blue = -20.36+5*np.log10(h) #from table 4
phiStar_blue = (10**(-2.27*(h**3))) #[Mpc^-3] --- from table 4
alpha_blue = -1.38 # from table 4

MStar_red = -20.68+5*np.log10(h) #from table 4
phiStar_red = (10**(-2.23*(h**3))) #[Mpc^-3] --- from table 4
alpha_red = -0.79 # from table 4


def schechter_lum(M, MStar, phiStar, alpha):
    """Schechter luminosity function with better numerical handling"""
    logL = 0.4 * (MStar - M)
    return phiStar * np.log(10) * (10**logL)**(alpha+1) * np.exp(-10**logL)

In [39]:
dM = 0.001
int_over = np.arange(-23, -14, dM)

n_MBlue = 0
for M in int_over:
    ## Approximating the integral as a summation
    n_MBlue += schechter_lum(M, MStar_blue, phiStar_blue, alpha_blue) * dM

print(f'{n_MBlue:.4} blue galaxies per Mpc³')
    
n_MRed = 0
for M in int_over:
    n_MRed += schechter_lum(M, MStar_red, phiStar_red, alpha_red) * dM

print(f'{n_MRed:.3} red galaxies per Mpc³')

11.73 blue galaxies per Mpc³
1.39 red galaxies per Mpc³


Unclear what makes my calculation 'local' as the question asks for. 

# 1.2

>  Make a plot showing how the local luminosity density of red and blue galaxies changes as a function of faint-end slope α, exploring the range -2.0 < α < -0.5. You may assume the Schechter function applies over an infinite luminosity range. (Note: luminosity density should be plotted in units of solar luminosity per cubic Mpc)
